In [146]:
import gurobipy as gp
from gurobipy import GRB

In [147]:
import pandas as pd
data = pd.read_excel('Top.xlsx')
data = data[['구간', '연장(km)', '차로수', '계']]
data.head()

,구간,연장(km),차로수,계
0,구서IC-노포IC,5.1,6,71512
1,노포IC-노포JCT,0.5,6,66817
2,노포JCT-양산JCT,7.3,6,86906
3,양산JCT-양산IC,5.1,8,89018
4,양산IC-통도사하이패스IC,12.8,6,71192


In [148]:
tmp = pd.DataFrame(data = [], columns = data.columns)

In [149]:
try:

    # Create a new model
    m = gp.Model("qp")

    # Create variables
    for i in range(1, len(data)+1):
        globals()['x_%d'%i] = m.addVar(vtype=GRB.BINARY, name="x_%d"%i)
        globals()['y_%d'%i] = m.addVar(vtype=GRB.CONTINUOUS, name="y_%d"%i)
    for i in range(len(data)):
        globals()['l_%d'%(i+1)] = data.loc[i, '연장(km)']
        globals()['n_%d'%(i+1)] = data.loc[i, '차로수']
        globals()['e_%d'%(i+1)] = data.loc[i, '계']

    # Set objective
    obj = None
    for i in range(1, len(data)+1):
        obj += globals()['x_%d'%i] * globals()['y_%d'%i] * globals()['e_%d'%i]
    m.setObjective(obj, GRB.MAXIMIZE)
    
    const = None
    for i in range(1, len(data)+1):
        const +=  globals()['x_%d'%i] * globals()['y_%d'%i] * globals()['n_%d'%i]
    # Add constraint: x + 2 y + 3 z <= 4
    m.addConstr(13.5 * const <= 5179, "c0")

    # Add constraint: x + y >= 1
    for i in range(1, len(data)+1):
        m.addConstr(globals()['y_%d'%i] <= globals()['l_%d'%i], "c_%d"%i)

    # Optimize model
    m.optimize()

    for v in m.getVars():
        print('%s %g' % (v.varName, v.x))

    print('Obj: %g' % m.objVal)

except gp.GurobiError as e:
    print('Error code ' + str(e.errno) + ': ' + str(e))

except AttributeError:
    print('Encountered an attribute error')

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 120 rows, 240 columns and 120 nonzeros
Model fingerprint: 0xe78a2b7c
Model has 120 quadratic objective terms
Model has 1 quadratic constraint
Variable types: 120 continuous, 120 integer (120 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [5e+01, 1e+02]
  Objective range  [0e+00, 0e+00]
  QObjective range [5e+04, 5e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-01, 2e+01]
  QRHS range       [5e+03, 5e+03]
Found heuristic solution: objective -0.0000000
Presolve removed 120 rows and 0 columns
Presolve time: 0.00s
Presolved: 361 rows, 360 columns, 960 nonzeros
Variable types: 240 continuous, 120 integer (120 binary)

Root relaxation: objective -1.015894e+07, 1 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth Int

In [150]:
lst = []
for v in m.getVars() :
    if "y" in v.varName :
        if v.x > 0 :
            lst.append(int(v.varName.split('_')[1])-1)

In [151]:
lst

[49, 51, 54, 56, 58, 59, 65, 66, 68, 69, 70, 71, 95]

In [152]:
tmp = pd.concat([data.loc[lst,], tmp])

In [153]:
tmp

,구간,연장(km),차로수,계
49,기흥IC-수원신갈IC,5.1,8,211996
51,신갈JCT-판교JCT,13.1,8,201226
54,판교JCT-성남IC,4.2,8,209014
56,송파IC-서하남IC,5.0,8,202516
58,하남JCT-상일IC,2.6,8,223519
59,상일IC-강일IC,3.0,8,239881
65,자유로JCT-김포IC,3.1,8,201740
66,김포IC-노오지JCT,3.8,8,189513
68,계양IC-서운JCT,2.3,8,212076
69,서운JCT-중동IC,2.1,8,253490


In [154]:
data.loc[lst,]

,구간,연장(km),차로수,계
49,기흥IC-수원신갈IC,5.1,8,211996
51,신갈JCT-판교JCT,13.1,8,201226
54,판교JCT-성남IC,4.2,8,209014
56,송파IC-서하남IC,5.0,8,202516
58,하남JCT-상일IC,2.6,8,223519
59,상일IC-강일IC,3.0,8,239881
65,자유로JCT-김포IC,3.1,8,201740
66,김포IC-노오지JCT,3.8,8,189513
68,계양IC-서운JCT,2.3,8,212076
69,서운JCT-중동IC,2.1,8,253490


In [155]:
for i in lst :
    if data.iloc[i,1] != globals()['y_%d'%(i+1)].x :
        data.iloc[i,1] -= globals()['y_%d'%(i+1)].x
        lst.remove(i)

In [156]:
data.drop(index = lst, inplace = True)
data

,구간,연장(km),차로수,계
0,구서IC-노포IC,5.1,6,71512
1,노포IC-노포JCT,0.5,6,66817
2,노포JCT-양산JCT,7.3,6,86906
3,양산JCT-양산IC,5.1,8,89018
4,양산IC-통도사하이패스IC,12.8,6,71192
...,...,...,...,...
115,면온IC-평창IC,5.7,4,34096
116,평창IC-속사IC,9.2,4,30591
117,속사IC-진부IC,7.2,4,29849
118,진부IC-대관령IC,12.8,4,28432


In [157]:
data.reset_index(drop=True, inplace= True)

In [158]:
try:

    # Create a new model
    m = gp.Model("qp")

    # Create variables
    for i in range(1, len(data)+1):
        globals()['x_%d'%i] = m.addVar(vtype=GRB.BINARY, name="x_%d"%i)
        globals()['y_%d'%i] = m.addVar(vtype=GRB.CONTINUOUS, name="y_%d"%i)
    for i in range(len(data)):
        globals()['l_%d'%(i+1)] = data.loc[i, '연장(km)']
        globals()['n_%d'%(i+1)] = data.loc[i, '차로수']
        globals()['e_%d'%(i+1)] = data.loc[i, '계']

    # Set objective
    obj = None
    for i in range(1, len(data)+1):
        obj += globals()['x_%d'%i] * globals()['y_%d'%i] * globals()['e_%d'%i]
    m.setObjective(obj, GRB.MAXIMIZE)
    
    const = None
    for i in range(1, len(data)+1):
        const +=  globals()['x_%d'%i] * globals()['y_%d'%i] * globals()['n_%d'%i]
    # Add constraint: x + 2 y + 3 z <= 4
    m.addConstr(13.5 * const <= 5179, "c0")

    # Add constraint: x + y >= 1
    for i in range(1, len(data)+1):
        m.addConstr(globals()['y_%d'%i] <= globals()['l_%d'%i], "c_%d"%i)

    # Optimize model
    m.optimize()

    for v in m.getVars():
        print('%s %g' % (v.varName, v.x))

    print('Obj: %g' % m.objVal)

except gp.GurobiError as e:
    print('Error code ' + str(e.errno) + ': ' + str(e))

except AttributeError:
    print('Encountered an attribute error')

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 108 rows, 216 columns and 108 nonzeros
Model fingerprint: 0x70604b7b
Model has 108 quadratic objective terms
Model has 1 quadratic constraint
Variable types: 108 continuous, 108 integer (108 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [5e+01, 1e+02]
  Objective range  [0e+00, 0e+00]
  QObjective range [5e+04, 4e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-01, 2e+01]
  QRHS range       [5e+03, 5e+03]
Found heuristic solution: objective -0.0000000
Presolve removed 108 rows and 0 columns
Presolve time: 0.00s
Presolved: 325 rows, 324 columns, 864 nonzeros
Variable types: 216 continuous, 108 integer (108 binary)

Root relaxation: objective -8.568853e+06, 1 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth Int

In [159]:
lst2 = []
for v in m.getVars() :
    if "y" in v.varName :
        if v.x > 0 :
            lst2.append(int(v.varName.split('_')[1])-1)

In [160]:
lst2

[43, 45, 47, 52, 53, 55, 56, 59, 60, 71, 76, 79, 82, 83, 84]

In [161]:
data.loc[lst2,]

,구간,연장(km),차로수,계
43,천안IC-북천안IC,8.200000,8,180070
45,안성IC-안성JCT,4.800000,8,188440
47,오산IC-동탄JCT,3.900000,8,169972
52,성남IC-송파IC,5.300000,8,175490
53,서하남IC-하남JCT,5.000000,8,174832
55,토평IC-남양주IC,2.400000,8,188115
56,남양주IC-구리IC,2.000000,8,166752
59,김포IC-노오지JCT,2.246296,8,189513
60,노오지JCT-계양IC,3.000000,8,170514
71,의정부IC-호원IC,1.600000,8,166803


In [162]:
for i in lst2 :
    if data.iloc[i,1] != globals()['y_%d'%(i+1)].x :
        data.iloc[i,1] -= globals()['y_%d'%(i+1)].x
        lst2.remove(i)

In [163]:
data.drop(index = lst2, inplace = True)
data

,구간,연장(km),차로수,계
0,구서IC-노포IC,5.1,6,71512
1,노포IC-노포JCT,0.5,6,66817
2,노포JCT-양산JCT,7.3,6,86906
3,양산JCT-양산IC,5.1,8,89018
4,양산IC-통도사하이패스IC,12.8,6,71192
...,...,...,...,...
103,면온IC-평창IC,5.7,4,34096
104,평창IC-속사IC,9.2,4,30591
105,속사IC-진부IC,7.2,4,29849
106,진부IC-대관령IC,12.8,4,28432


In [164]:
data.reset_index(drop=True, inplace= True)

In [165]:
try:

    # Create a new model
    m = gp.Model("qp")

    # Create variables
    for i in range(1, len(data)+1):
        globals()['x_%d'%i] = m.addVar(vtype=GRB.BINARY, name="x_%d"%i)
        globals()['y_%d'%i] = m.addVar(vtype=GRB.CONTINUOUS, name="y_%d"%i)
    for i in range(len(data)):
        globals()['l_%d'%(i+1)] = data.loc[i, '연장(km)']
        globals()['n_%d'%(i+1)] = data.loc[i, '차로수']
        globals()['e_%d'%(i+1)] = data.loc[i, '계']

    # Set objective
    obj = None
    for i in range(1, len(data)+1):
        obj += globals()['x_%d'%i] * globals()['y_%d'%i] * globals()['e_%d'%i]
    m.setObjective(obj, GRB.MAXIMIZE)
    
    const = None
    for i in range(1, len(data)+1):
        const +=  globals()['x_%d'%i] * globals()['y_%d'%i] * globals()['n_%d'%i]
    # Add constraint: x + 2 y + 3 z <= 4
    m.addConstr(13.5 * const <= 5179, "c0")

    # Add constraint: x + y >= 1
    for i in range(1, len(data)+1):
        m.addConstr(globals()['y_%d'%i] <= globals()['l_%d'%i], "c_%d"%i)

    # Optimize model
    m.optimize()

    for v in m.getVars():
        print('%s %g' % (v.varName, v.x))

    print('Obj: %g' % m.objVal)

except gp.GurobiError as e:
    print('Error code ' + str(e.errno) + ': ' + str(e))

except AttributeError:
    print('Encountered an attribute error')

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 94 rows, 188 columns and 94 nonzeros
Model fingerprint: 0xcb34b732
Model has 94 quadratic objective terms
Model has 1 quadratic constraint
Variable types: 94 continuous, 94 integer (94 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [5e+01, 1e+02]
  Objective range  [0e+00, 0e+00]
  QObjective range [5e+04, 4e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-01, 2e+01]
  QRHS range       [5e+03, 5e+03]
Found heuristic solution: objective -0.0000000
Presolve removed 94 rows and 0 columns
Presolve time: 0.00s
Presolved: 283 rows, 282 columns, 752 nonzeros
Variable types: 188 continuous, 94 integer (94 binary)

Root relaxation: objective -7.785097e+06, 1 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Inc

In [166]:
lst3 = []
for v in m.getVars() :
    if "y" in v.varName :
        if v.x > 0 :
            lst3.append(int(v.varName.split('_')[1])-1)

In [167]:
data.loc[lst3,]

,구간,연장(km),차로수,계
43,북천안IC-안성IC,11.800000,8,157440
44,안성JCT-오산IC,13.300000,8,166623
45,동탄JCT-기흥IC,5.100000,8,159599
46,수원신갈IC-신갈JCT,2.700000,10,197800
50,남양주IC-구리IC,1.967593,8,166752
51,구리IC-퇴계원IC,2.800000,8,161442
56,도리JCT-조남JCT,4.400000,8,162491
60,학의JCT-판교JCT,8.800000,8,158708
66,고양IC-일산IC,3.300000,8,161999
70,안산JCT-둔대JCT,4.800000,6,122552


In [168]:
for i in lst3 :
    if data.iloc[i,1] != globals()['y_%d'%(i+1)].x :
        data.iloc[i,1] -= globals()['y_%d'%(i+1)].x
        lst3.remove(i)

In [169]:
data.drop(index = lst3, inplace = True)
data

,구간,연장(km),차로수,계
0,구서IC-노포IC,5.1,6,71512
1,노포IC-노포JCT,0.5,6,66817
2,노포JCT-양산JCT,7.3,6,86906
3,양산JCT-양산IC,5.1,8,89018
4,양산IC-통도사하이패스IC,12.8,6,71192
...,...,...,...,...
89,면온IC-평창IC,5.7,4,34096
90,평창IC-속사IC,9.2,4,30591
91,속사IC-진부IC,7.2,4,29849
92,진부IC-대관령IC,12.8,4,28432


In [170]:
data.reset_index(drop=True, inplace= True)

In [171]:
try:

    # Create a new model
    m = gp.Model("qp")

    # Create variables
    for i in range(1, len(data)+1):
        globals()['x_%d'%i] = m.addVar(vtype=GRB.BINARY, name="x_%d"%i)
        globals()['y_%d'%i] = m.addVar(vtype=GRB.CONTINUOUS, name="y_%d"%i)
    for i in range(len(data)):
        globals()['l_%d'%(i+1)] = data.loc[i, '연장(km)']
        globals()['n_%d'%(i+1)] = data.loc[i, '차로수']
        globals()['e_%d'%(i+1)] = data.loc[i, '계']

    # Set objective
    obj = None
    for i in range(1, len(data)+1):
        obj += globals()['x_%d'%i] * globals()['y_%d'%i] * globals()['e_%d'%i]
    m.setObjective(obj, GRB.MAXIMIZE)
    
    const = None
    for i in range(1, len(data)+1):
        const +=  globals()['x_%d'%i] * globals()['y_%d'%i] * globals()['n_%d'%i]
    # Add constraint: x + 2 y + 3 z <= 4
    m.addConstr(13.5 * const <= 5179, "c0")

    # Add constraint: x + y >= 1
    for i in range(1, len(data)+1):
        m.addConstr(globals()['y_%d'%i] <= globals()['l_%d'%i], "c_%d"%i)

    # Optimize model
    m.optimize()

    for v in m.getVars():
        print('%s %g' % (v.varName, v.x))

    print('Obj: %g' % m.objVal)

except gp.GurobiError as e:
    print('Error code ' + str(e.errno) + ': ' + str(e))

except AttributeError:
    print('Encountered an attribute error')

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 85 rows, 170 columns and 85 nonzeros
Model fingerprint: 0x112e7171
Model has 85 quadratic objective terms
Model has 1 quadratic constraint
Variable types: 85 continuous, 85 integer (85 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [5e+01, 1e+02]
  Objective range  [0e+00, 0e+00]
  QObjective range [5e+04, 4e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-01, 2e+01]
  QRHS range       [5e+03, 5e+03]
Found heuristic solution: objective -0.0000000
Presolve removed 85 rows and 0 columns
Presolve time: 0.00s
Presolved: 256 rows, 255 columns, 680 nonzeros
Variable types: 170 continuous, 85 integer (85 binary)

Root relaxation: objective -7.348395e+06, 1 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Inc

In [172]:
lst4 = []
for v in m.getVars() :
    if "y" in v.varName :
        if v.x > 0 :
            lst4.append(int(v.varName.split('_')[1])-1)

In [173]:
data.loc[lst4,]

,구간,연장(km),차로수,계
42,천안JCT-천안IC,6.700000,8,148557
43,북천안IC-안성IC,10.488889,8,157440
47,일산IC-자유로JCT,2.200000,10,189032
48,장수IC-시흥IC,4.200000,8,155771
49,시흥IC-안현JCT,2.700000,8,152620
51,조남JCT-산본IC,5.300000,8,154501
53,평촌IC-학의JCT,2.900000,8,153431
56,호원IC-송추IC,6.500000,8,154385
57,송추IC-통일로IC,8.900000,8,148438


In [174]:
for i in lst4 :
    if data.iloc[i,1] != globals()['y_%d'%(i+1)].x :
        data.iloc[i,1] -= globals()['y_%d'%(i+1)].x
        lst4.remove(i)

In [175]:
/0data.drop(index = lst4, inplace = True)
data

,구간,연장(km),차로수,계
0,구서IC-노포IC,5.1,6,71512
1,노포IC-노포JCT,0.5,6,66817
2,노포JCT-양산JCT,7.3,6,86906
3,양산JCT-양산IC,5.1,8,89018
4,양산IC-통도사하이패스IC,12.8,6,71192
...,...,...,...,...
80,면온IC-평창IC,5.7,4,34096
81,평창IC-속사IC,9.2,4,30591
82,속사IC-진부IC,7.2,4,29849
83,진부IC-대관령IC,12.8,4,28432


In [176]:
data.reset_index(drop=True, inplace= True)

In [177]:
try:

    # Create a new model
    m = gp.Model("qp")

    # Create variables
    for i in range(1, len(data)+1):
        globals()['x_%d'%i] = m.addVar(vtype=GRB.BINARY, name="x_%d"%i)
        globals()['y_%d'%i] = m.addVar(vtype=GRB.CONTINUOUS, name="y_%d"%i)
    for i in range(len(data)):
        globals()['l_%d'%(i+1)] = data.loc[i, '연장(km)']
        globals()['n_%d'%(i+1)] = data.loc[i, '차로수']
        globals()['e_%d'%(i+1)] = data.loc[i, '계']

    # Set objective
    obj = None
    for i in range(1, len(data)+1):
        obj += globals()['x_%d'%i] * globals()['y_%d'%i] * globals()['e_%d'%i]
    m.setObjective(obj, GRB.MAXIMIZE)
    
    const = None
    for i in range(1, len(data)+1):
        const +=  globals()['x_%d'%i] * globals()['y_%d'%i] * globals()['n_%d'%i]
    # Add constraint: x + 2 y + 3 z <= 4
    m.addConstr(13.5 * const <= 5179, "c0")

    # Add constraint: x + y >= 1
    for i in range(1, len(data)+1):
        m.addConstr(globals()['y_%d'%i] <= globals()['l_%d'%i], "c_%d"%i)

    # Optimize model
    m.optimize()

    for v in m.getVars():
        print('%s %g' % (v.varName, v.x))

    print('Obj: %g' % m.objVal)

except gp.GurobiError as e:
    print('Error code ' + str(e.errno) + ': ' + str(e))

except AttributeError:
    print('Encountered an attribute error')

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 77 rows, 154 columns and 77 nonzeros
Model fingerprint: 0x5dad1eb5
Model has 77 quadratic objective terms
Model has 1 quadratic constraint
Variable types: 77 continuous, 77 integer (77 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [5e+01, 1e+02]
  Objective range  [0e+00, 0e+00]
  QObjective range [5e+04, 4e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-01, 2e+01]
  QRHS range       [5e+03, 5e+03]
Found heuristic solution: objective -0.0000000
Presolve removed 77 rows and 0 columns
Presolve time: 0.00s
Presolved: 232 rows, 231 columns, 616 nonzeros
Variable types: 154 continuous, 77 integer (77 binary)

Root relaxation: objective -6.891600e+06, 1 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Inc

In [178]:
lst5 = []
for v in m.getVars() :
    if "y" in v.varName :
        if v.x > 0 :
            lst5.append(int(v.varName.split('_')[1])-1)

In [179]:
data.loc[lst5,]

,구간,연장(km),차로수,계
17,북대구IC-금호JCT,5.500000,8,144977
43,금토JCT-양재IC,6.800000,10,181871
44,강일IC-토평IC,1.300000,8,140100
45,안현JCT-도리JCT,4.800000,8,142884
46,산본IC-평촌IC,4.100000,8,136274
48,별내IC-의정부IC,8.000000,8,141331
49,송추IC-통일로IC,2.485185,8,148438
50,통일로IC-고양IC,6.000000,8,147078
54,북수원IC-동수원IC,6.100000,8,143044
55,동수원IC-신갈JCT,4.900000,8,139741
